In [6]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.svm import SVR
from scipy.stats import uniform, randint
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('data/train_processed.csv')
X_train, y_train = df.drop(columns=['SalePrice']), df['SalePrice']
X_test = pd.read_csv('data/test-processed.csv')

In [ ]:
params = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'] ,
    'degree': randint(1, 10),
    'gamma': ['scale', 'auto'],
    'C': uniform(0.1, 10)
}
svr = SVR()
random_search = RandomizedSearchCV(
    estimator=svr,
    param_distributions=params,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5
)
random_search.fit(X_train, y_train)

In [ ]:
best_model = random_search.best_estimator_

scores = cross_val_score(best_model, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
scores = np.sqrt(-scores)

In [ ]:
predictions = pd.DataFrame({'Id': X_test['Id'], 'SalePrice': best_model.predict(X_test)})
predictions.set_index('Id', inplace=True)
predictions.to_csv('data/predictions/svr.csv')